#### Necessary packages

In [468]:
# numeric and plotting libraries
import time
import os
import glob
import math
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import csv
import random
from PIL import Image
import csv

In [469]:
# deep learning/vision libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets, models, transforms

In [470]:
# use GPU if it's available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## CNN classification

- 1 region, all info about speakers, 4 sessions, white noise, 50 info per neuron (per speaker), 100 neurons per region => image 100x250(5x50) - 8 of them (2 states x 4 sessions) x 30 (sampling points over time)
- final 240 images (120 per state)
- oversampling input images with NaNs (maybe random oversampling)
- combination of 2 different regions in one image/mice???
- including bandpass noise into data representation 
- representing input images as different stimulus (100 x 50)
---
- training on combined sessions, different sessions (evaluating results on different sessions)
- maybe training on the particular stimulus and then seeing wich stimulus is the most inforative for the state classification
- performing classification on different regions to see which region (to see if some regions have very noninformative responses)
- performin classification on different combination of regions???
- comparing the classificator performance - input data is white noise/bandpass noise/combination of noises
---
- stimulus classificators

### 1. Classification between anest and awake state

#### Data organization

In [471]:
def ucitavanje_podataka(directory):
    id = 0
    data = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        label = f.split("\\")[-1][0:2]
        id = id + 1
        dat = np.load(f)
        data.append([id, dat, label])
    
    random.seed(2)
    random.shuffle(data)
    return data        

In [472]:
def vektorizacija(podaci):
    prevodjenje_labela = {"an": [1], "aw": [0]}  # TRUE:Mirtrons  FALSE:canonical microRN

    vektorizovani_podaci = []
    for a in podaci:
        vektorizovani_podaci.append([a[0], a[1], prevodjenje_labela[a[2]]])

    return vektorizovani_podaci

In [473]:
def podela_podataka(data_vectors):
    X_train, y_train, X_test, y_test = [],[],[],[]
    i,j = 0,0

    random.shuffle(data_vectors)
    for item in data_vectors:
        if item[2]==[1]:
            i = i + 1
            if i <= 50:
                X_test.append(item[1])
                y_test.append(item[2])
            else:
                X_train.append(item[1])
                y_train.append(item[2])
        else:
            j = j + 1
            if  j<= 50:
                X_test.append(item[1])
                y_test.append(item[2])
            else:
                X_train.append(item[1])
                y_train.append(item[2])

    return X_train, y_train, X_test, y_test

In [486]:
putanja = r"C:\Users\m.nedeljkovic\Desktop\cnn data\data"
#niz od podataka za svaku sekvencu (podatak je niz od id-ja, sekvence i labele)
podaci = ucitavanje_podataka(putanja)

vektorizovani_podaci = vektorizacija(podaci)
X_train, y_train, X_test, y_test = podela_podataka(vektorizovani_podaci)

In [527]:
X_train = np.array(X_train).reshape(300, 1, 110, 155)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(100, 1, 110, 155)
y_test = np.array(y_test)

In [528]:
X_test2 = X_test[:20]
y_test2 = y_test[:20]

X_test1 = X_test[20:]
y_test1 = y_test[20:]

In [529]:
savеpath = "C:/Users/m.nedeljkovic/Desktop/cnn data/"

for i in range(len(X_train)):
    np.save(savеpath + 'train/' + str(y_train[i][0]) + '/' + str(i) + '.npy', X_train[i])

for i in range(len(X_test1)):
    with open(savеpath + 'val/' + str(y_test1[i][0]) + '/' + str(i) + '.npy', 'wb') as f:
        np.save(f, X_test1[i])

for i in range(len(X_test2)):
    with open(savеpath + 'test/' + str(y_test2[i][0]) + '/' + str(i) + '.npy', 'wb') as f:
        np.save(f, X_test2[i])

#### Hyperparameters

In [588]:
# hyperparameters
BATCH_SIZE = 16  # batch size of input

# CONV_SIZE = 3    #first filter size
# CONV_DEEP = 128   #number of first filter(convolution deepth)

DROPOUT_KEEP_PROB = 0.5  # keep probability of dropout

#### Dataset loader

In [560]:
class NumpyDataset(Dataset):

    def __init__(self, data_numpy_list, data_transforms):
        self.data_numpy_list = data_numpy_list
        self.transform = data_transforms
        self.data_list = []
        self.label_list = []
        for ind in range(len(self.data_numpy_list)):
            data_slice_file_name = self.data_numpy_list[ind]
            data_i = np.load(data_slice_file_name)
            idx = data_slice_file_name.rfind("\\")
            self.data_list.append(data_i)
            self.label_list.append(int(data_slice_file_name[idx-1]))

    def __getitem__(self, index):

        data = np.asarray(self.data_list[index])
        label = np.asarray(self.label_list[index])
        data = torch.from_numpy(data)
        label = torch.from_numpy(label)
        return data, label

    def __len__(self):
        return len(self.data_numpy_list)


In [561]:
data_transforms = {
    transforms.Compose([
        transforms.ToTensor(),
    ])
}

#### Convolutional neural network

In [594]:
# define placeholder
class CustomCNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.relu = torch.nn.ReLU()
        
        self.conv1 = nn.Conv2d(1, 16, (3, 3), stride=1)
        self.conv2 = nn.Conv2d(16, 64, (3, 3), stride=1)
        self.conv3 = nn.Conv2d(64, 128, (3, 3), stride=1)
        self.conv4 = nn.Conv2d(128, 128, (3, 3), stride=1)
        self.conv5 = nn.Conv2d(128, 256, (3, 3), stride=1)
        self.conv6 = nn.Conv2d(256, 256, (3, 3), stride=1)
        self.conv7 = nn.Conv2d(256, 512, (3, 3), stride=1)
        self.conv8 = nn.Conv2d(512, 512, (3, 3), stride=1)

        self.norm1 = nn.BatchNorm2d(16)
        self.norm2 = nn.BatchNorm2d(64)
        self.norm3 = nn.BatchNorm2d(128)
        self.norm4 = nn.BatchNorm2d(256)
        self.norm5 = nn.BatchNorm2d(512)

        self.maxpool = nn.MaxPool2d((2, 2))
      
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
       

        self.sf = nn.Softmax(1)

    def forward(self, x):
       
        y1 = self.conv1(x)
        y1 = self.norm1(y1)
        y2 = self.conv2(y1)
        y3 = self.maxpool(y2)
        y4 = self.conv3(y3)
        y4 = self.norm3(y4)
        y5 = self.conv4(y4)
        y6 = self.maxpool(y5)
        y7 = self.conv5(y6)
        y7 = self.norm4(y7)
        y8 = self.conv6(y7)
        y9 = self.maxpool(y8)
        y10 = self.conv7(y9)
        y11 = self.conv8(y10)
        y12 = self.conv8(y11)
        y13 = self.maxpool(y12)
        y14 = torch.flatten(y13, 1)
        y15 = self.fc1(y14)
        y16 = self.fc2(y15)
        y17 = self.fc3(y16)
        y = self.sf(y17)

        return y

#### Model training

In [583]:
def train_model(model, criterion, optimizer, num_epochs, path, dataloaders):
    start_time = time.time()

    metrics = defaultdict(list)
    total = 0
    correct = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            best_acc = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, labels.long())
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                for i in range(len(preds)):
                    if(preds[i] == labels.data[i]):
                        if(preds[i] == 1):
                            tp += 1
                        else:
                            tn += 1
                    else:
                        if (preds[i] == 1):
                            fp += 1
                        else:
                            fn += 1

                # statistics
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).item()

                total += labels.size(0)
                correct += (preds == labels.data).sum()

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = 100 * float(correct) / total

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc

            metrics[phase + "_loss"].append(epoch_loss)
            metrics[phase + "_acc"].append(epoch_acc)


    time_elapsed = time.time() - start_time
    print(f'Training complete in {(time_elapsed // 60):.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc: .4f}')

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = 2 * tp / (2 * tp + fp + fn)
    print(f'Sens: {sensitivity: .4f} Spec: {specificity: .4f} F1: {f1: .4f}')

    torch.save(model.state_dict(), path)
    # load best model weights
    return model, metrics

#### Evaluating model

In [564]:
def model_eval(model, criterion ,optimizer, PATH, podaci):
    model.load_state_dict(torch.load(PATH))
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    correct = 0
    tp=0
    tn=0
    fp=0
    fn=0
    phase = 'test'
    for inputs, labels in podaci[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs.float())
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels.long())
        for i in range(len(preds)):
            if (preds[i] == labels.data[i]):
                if (preds[i] == 1):
                    tp += 1
                else:
                    tn += 1
            else:
                if (preds[i] == 1):
                    fp += 1
                else:
                    fn += 1

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

        total += labels.size(0)
        correct += (preds == labels.data).sum()

    epoch_loss = running_loss / 201
    epoch_acc = 100 * float(correct) / total
    mcc = (tn * tp-fp*fn) / math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)) 
    print(f'MCC: {mcc: .4f}')
    print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return epoch_acc


#### Count parameters

In [565]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#### Main

In [595]:
if __name__ == "__main__":
    putanja = "C:/Users/m.nedeljkovic/Desktop/cnn data/data"
    #niz od podataka za svaku sekvencu (podatak je niz od id-ja, sekvence i labele)
    podaci = ucitavanje_podataka(putanja)

    vektorizovani_podaci = vektorizacija(podaci)
    X_train, y_train, X_test, y_test = podela_podataka(vektorizovani_podaci)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    X_test2 = X_test[:20]
    y_test2 = y_test[:20]

    X_test1 = X_test[20:]
    y_test1 = y_test[20:]

    dataset_sizes = {'train': len(X_train),
                    'val': len(X_test1),
                     'test': len(X_test2)}

    dataset = {
        'train': [X_train, y_train],
        'val': [X_test, y_test]
    }

    data_dir = "C:/Users/m.nedeljkovic/Desktop/cnn data"
    path1 = "C:/Users/m.nedeljkovic/Desktop/Model/state_dict_model.pt"

    # basic error checking to check whether you correctly unzipped the dataset into the working directory
    assert os.path.exists(data_dir), f'Could not find {data_dir} in working directory {os.getcwd()}n'
    dirs_exist = [os.path.exists(os.path.join(data_dir, x)) for x in ['train', 'val']]
    assert all(dirs_exist), f'Could not find train/val dirs check if you have train and val directly under {data_dir}.'
    data_numpy_list = {}
    data_numpy_list['train'] = [x for x in glob.glob(os.path.join(data_dir, 'train/**/*.npy'), recursive=True)]
    data_numpy_list['val'] = [x for x in glob.glob(os.path.join(data_dir, 'val/**/*.npy'), recursive=True)]
   
    # ImageFolder is a PyTorch class - it expects <class1-name>, <class2-name>, ...folders under the root path you give it
    datasets = {x: NumpyDataset(data_numpy_list[x], data_transforms) for x in ['train', 'val']}
    dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['train', 'val']}
    criterion = nn.CrossEntropyLoss()

    custom_cnn = CustomCNN().to(device)
    optimizer_conv = optim.Adam(filter(lambda p: p.requires_grad, custom_cnn.parameters()))

    print(f"number of params in model {count_parameters(custom_cnn)}")
    model_conv, metrics = train_model(custom_cnn, criterion, optimizer_conv, num_epochs=100, path=path1, dataloaders=dataloaders)

    train_loss = np.array(metrics['train_loss'])
    val_loss = np.array(metrics['val_loss'])
    train_acc = np.array(metrics['train_acc'])
    val_acc = np.array(metrics['val_acc'])

    np.savetxt('train_loss.csv', train_loss, delimiter=',')
    np.savetxt('val_loss.csv', val_loss, delimiter=',')
    np.savetxt('train_acc.csv', train_acc, delimiter=',')
    np.savetxt('val_acc.csv', val_acc, delimiter=',')

    podaci_za_test = [x for x in glob.glob(os.path.join(data_dir, 'test/**/*.npy'), recursive=True)]
    datasets1 = {x: NumpyDataset(podaci_za_test, data_transforms) for x in ['test']}
    podaci = {x: torch.utils.data.DataLoader(datasets1[x], batch_size=BATCH_SIZE, shuffle=True) for x in
              ['test']}

number of params in model 9411008
Epoch 0/99
----------
train Loss: nan Acc: 50.0000
val Loss: nan Acc: 50.7895
Epoch 1/99
----------
train Loss: nan Acc: 50.4412
val Loss: nan Acc: 50.7895
Epoch 2/99
----------
train Loss: nan Acc: 50.5660
val Loss: nan Acc: 50.7895
Epoch 3/99
----------
train Loss: nan Acc: 50.6250
val Loss: nan Acc: 50.7895
Epoch 4/99
----------
train Loss: nan Acc: 50.6593
val Loss: nan Acc: 50.7895
Epoch 5/99
----------
train Loss: nan Acc: 50.6818
val Loss: nan Acc: 50.7895
Epoch 6/99
----------
train Loss: nan Acc: 50.6977
val Loss: nan Acc: 50.7895
Epoch 7/99
----------
train Loss: nan Acc: 50.7095
val Loss: nan Acc: 50.7895
Epoch 8/99
----------
train Loss: nan Acc: 50.7186
val Loss: nan Acc: 50.7895
Epoch 9/99
----------
train Loss: nan Acc: 50.7258
val Loss: nan Acc: 50.7895
Epoch 10/99
----------
train Loss: nan Acc: 50.7317
val Loss: nan Acc: 50.7895
Epoch 11/99
----------
train Loss: nan Acc: 50.7366
val Loss: nan Acc: 50.7895
Epoch 12/99
----------
train

KeyboardInterrupt: 